# Process base variables

This Jupyter Notebook processes raw CMIP6 climate model data for a list of specified variables. 
It calculates annual means or sums (depending on the variable) for each variable, optionally retaining the spatial dimension.

It is set up to process variables with the following units:
- Flux Variables (2D fields in [kg m-2 s-1]) are converted as the sum over a year to: Pg yr-1 (1D), kg m-2 yr-1 (2D)
- Mol Flux Variables (2D fields in [mol m-2 s-1]) are converted  as the sum over a year to: mol yr-1 (1D), mol m-2 yr-1 (2D)
- Storage Variables (2D fields in [kg m-2] are converted as the average over a year to:  Pg (1D), kg m-2 (2D)
  
Requirements
- raw CMIP6 data
- grid-area files named ```{model}_{gridarea_var}_{gridtype}*``` for each model (e.g. MPI-ESM1-2-LR_areacello_gn.nc)

<img src="ArcticCarbon.png" alt="drawing" width="500"/>

## Setup

### Import Libraries & Catalog

In [1]:
# Basics
import numpy as np
import glob
import os
import sys

# Data Handling
import xarray as xr
import intake

# Processing
from cdo import Cdo
from tqdm import tqdm 
import logging
import re

# Load own functions
import functions.custom_logger_functions as lgfct
import functions.search_cmip6_catalog as searching

cdo = Cdo()

# load catalog
catalog = intake.open_esm_datastore("/work/ik1017/Catalogs/dkrz_cmip6_disk.json") # This takes a while to load...

print("DONE")

/sw/spack-levante/mambaforge-22.9.0-2-Linux-x86_64-kptncg/lib/python3.10/site-packages/intake_esm/cat.py:264: DtypeWarning: Columns (21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


DONE


### Set Parameters

In [2]:
# Select dimension of output data
dimensions = "1D" # "1D" or "2D"
time = "year" # "year" for annual data and "mon" for monthly data

# Create a list of variables for processing
ocean_vars = ["fgco2", "froc", "fric", "intpp", "intpoc", "intdic", "intdoc", "ocfriver", "fgco2"]
land_vars  = ["cLitter", "cSoil", "cVeg", "fLuc", "gpp", "nbp", "nep", "npp", "ra", "rh", "fFire"]
variables  = land_vars + ocean_vars
variables = ["fFire"]

scenarios = ["historical", "ssp126", "ssp245", "ssp370", "ssp585"]

# Select region
masked = True  # masked region only (Arctic Ocean or Permafrost mask defined below)
earth  = False # global mean
N60    = False # above 60°N only


# CREATE TERMINAL LOGGER that prints information based on the logging level
# logger will always print the statements according to the level plus all information from higher levels
#    e.g. logging level "info" will print all "info", "warning" and "error" text but not "debug"
#
# debug:   all print statement like file names from grid area, raw data ... or CDO command (great for debugging :) )
# info:    nice to know prints to see how the process is going (printing model, ensemble member, ...)
# warning: can't find files from the same ensemble member when combining variables
# error:   something is seriously wrong eg. variable name is not defined in the processing chain or a file should be there but isn't

logging_level = "warning"
logger = lgfct.build_terminal_logger(logging_level, logger_name="processing")

### Set Paths

In [8]:
outpath        = f"/work/uo1227/u301557/ArcticCarbon/data/{dimensions}/"                        # test outpath

#gridpath       = '/work/uo1227/DATA/modelling/CMIP6/gridareas/'                                # where the gridarea files are strored
gridpath       = '/work/uo1227/u301557/ArcticCarbon/data/gridarea/'


AO_mask        = "/home/u/u301557/ArcticCarbon/Arctic_ocean_mask_regions.nc"                    # Arctic Ocean Mask
land_mask      = "/work/uo1227/u301557/ArcticCarbon/data/orig_masks/permafrost_region_mask2.nc" # permafrost region mask
mask_var_names = {"ocean":"arctic_mask", "land":"mask"} # name of the variable in the dataset containing the mask

ocean_maskpath = "/work/uo1227/u301557/ArcticCarbon/data/ocean-masks-remap/"                    # where the remapped to the model grid masks will be stored
land_maskpath  = "/work/uo1227/u301557/ArcticCarbon/data/land-masks-remap/"

# !!! The masks will be remapped to each model grid. 
#     To save computing time the code will NOT perform the remapping each time but only if the remapped file is not available
#     If you change the mask here you have to delete all remapped mask files so that they will be calculated again from the new masks

## Processing Functions

In [9]:
def file_existence(filepath: str, logger: logging.Logger):
    """
    Checks if a file exists and logs a message accordingly.

    Parameters:
    - filepath (str): The path to the file to check.
    - logger (logging.Logger): The logger object to use for logging messages.

    """

    filename = filepath.split("/")[-1]

    if os.path.isfile(filepath):
        logger.debug(f"File '{filename}' exists.")
    else:
        logger.error(f"File '{filename}' does not exist.")

In [10]:
# Define CDO command based on variable type
def get_cdo_cmd_1D(variable: str, correction: str, area_file: str, inputfile: str, gridarea_var: str, time: str):
    """
    Generate a CDO command string based on the unit / variable type.

    Parameters:
    - variable (str): The variable for which to generate the CDO command.
    - correction (str): The correction to apply to the variable.
    - area_file (str): The file containing grid area information.
    - inputfile (str): The input file to process.
    - gridarea_var (str): grid name of the data (either areacello or areacella).
    - time (str): identifier for annual or monthly processing

    Returns:
    - str: The constructed CDO command, or None if the variable is not defined.
    """      

    if time == "year":
        unit_time = "yr-1"
        
    elif time == "mon":
        unit_time = "mon-1"

    # Flux Variables (2D fields in [kg m-2 s-1]) 
    if unit == "kg m-2 s-1":
        return (f" -setattribute,{variable}@units='Pg {unit_time}' -chname,{gridarea_var},{variable}{correction} "
                f"-divc,1e12 -{time}sum -mulc,2592000 -mul {area_file} -mergetime {inputfile}") # 2592000 = time conversion assuming 30 days in a month and monthly input data
        
    # Mol Flux Variables (2D fields in [mol m-2 s-1])
    if unit == "mol m-2 s-1":          
        return (f" -setattribute,{variable}@units='mol {unit_time}' -chname,{gridarea_var},{variable}{correction} "
                f"-{time}sum -mulc,2592000 -mul {area_file} -mergetime {inputfile}")
    
    # Storage Variables (2D fields in [kg m-2]) 
    elif unit == "kg m-2": 
        return (f" -setattribute,{variable}@units='Pg' -chname,{gridarea_var},{variable} "
                f"-{time}mean -divc,1e12 -mul {area_file} -mergetime {inputfile}")
    else:
        logger.error(f"Processing not defined for {variable}")
        return None


In [11]:
# Define CDO command based on variable type
def get_cdo_cmd_2D(variable, correction, inputfile, gridarea_var, time):
    """
    Generate a CDO command string based on the unit / variable type.

    Parameters:
    - variable (str): The variable for which to generate the CDO command.
    - correction (str): The correction to apply to the variable.
    - inputfile (str): The input file to process.
    - gridarea_var (str): grid name of the data (either areacello or areacella).
    - time (str): identifier for annual or monthly processing

    Returns:
    - str: The constructed CDO command, or None if the variable is not defined.
    """        

    if time == "year":
        unit_time = "yr-1"
        
    elif time == "mon":
        unit_time = "mon-1"
    
    # Flux Variables (2D fields in [kg m-2 s-1]) 
    if unit == "kg m-2 s-1": 
        return (f" -setattribute,{variable}@units='kg m-2 {unit_time}' {correction} "
                f" -{time}sum -mulc,2592000 -mergetime {inputfile}")
        
    # Mol Flux Variables (2D fields in [mol m-2 s-1])
    elif unit == "mol m-2 s-1": 
        return (f" -setattribute,{variable}@units='mol m-2 {unit_time}' {correction} "
                f" -{time}sum -mulc,2592000 -mergetime {inputfile}")
    
    # Storage Variables (2D fields in [kg m-2]) 
    elif unit == "kg m-2": 
        return (f" -{time}mean -mergetime {inputfile}")
    else:
        logger.error(f"Processing not defined for {variable}")
        return None

In [12]:
def retrieve_file_names(model: str, modelcenter: str, ensemblemember: str, scenario: str, variable: str, table_id: str, activity_id: str):
    """
    Constructs file paths and retrieves the names of input and area file for a specified CMIP6 model, ensemble member and variable.

    Parameters:
        model (str): The name of the CMIP6 model.
        modelcenter (str): The center or institution responsible for the model.
        ensemblemember (str): The ensemble member identifier (e.g., "r1i1p1f1").
        scenario (str): The emission scenario or experiment (e.g., "ssp585").
        variable (str): The variable of interest (e.g., "fgco2").
        table_id (str): The CMIP6 table identifier (e.g., "Amon").
        activity_id (str): The activity or project ID (e.g., "CMIP").

    Returns:
        - inputfile (str or None): File path to the NetCDF input file(s) matching the provided criteria, or None if the file is missing.
        - area_file (str or None): File path to the area data file, or None if the area file is missing.

    Notes:
        - The function searches for available grid types and versions within the constructed directory.
        - If the "v1" version is available, it prioritizes it; otherwise, it selects the highest available version.
    """
    
    # Construct filepath
    inpath = f"/pool/data/CMIP6/data/{activity_id}/{modelcenter}/{model}/{scenario}/{ensemblemember}/{table_id}/{variable}/" 
    logger.debug(f"inpath:     {inpath}")

    # Check if files exist in path
    folders_in_path = [os.path.basename(x) for x in glob.glob(inpath + '/*')]
    if not folders_in_path:
        logger.error(f"Filepath doesn't exist ({inpath})")
        return None, None
    
    # Get grid type and available versions
    gridtype = folders_in_path[0]
    av_versions = [os.path.basename(x) for x in glob.glob(f"{inpath}{gridtype}/v*")]
    
    # Determine version
    if 'v1' in av_versions:
        version = 'v1'
    else:
        version = 'v'+str(np.max([int(v[1::]) for v in av_versions]))
    
    inputfile = f"{inpath}{gridtype}/{version}/*.nc"

    # Find grid area file
    try:
        area_file = glob.glob(gridpath + f"{model}_{gridarea_var}_{gridtype}*")[0]
        logger.debug("area file:  " +area_file)
    except:
        logger.error(f"area file is missing: {gridpath}{model}_{gridarea_var}_{gridtype}*")
        area_file = None
        
    logger.debug("input file: " +inputfile)

    return inputfile, area_file

In [13]:
def cal_annual_monthly_values(inputfile: str, area_file: str, model: str, ensemblemember: str, scenario: str, variable: str, unit: str, time: str):

    """
    Calculates annual values for a specified variable using CDO and processes outputs for 
    global and regional domains.

    Parameters:
        inputfile (str): File path to the input NetCDF file containing the variable of interest.
        area_file (str): File path to the grid area data file.
        model (str): The name of the CMIP6 model (e.g., "ACCESS-CM2").
        ensemblemember (str): The ensemble member identifier (e.g., "r1i1p1f1").
        scenario (str): The emission scenario or experiment (e.g., "ssp585").
        variable (str): The variable to process (e.g., "fgco2").
        unit (str): The unit of the variable (e.g., "kg m-2 s-1").

    Returns:
        None

    Notes:
        - Constructs and executes CDO commands for spatial and temporal processing of the variable.
        - Handles different spatial dimensions ("1D" or "2D") and regional masks, including global and Arctic areas.
        - For masked regions, dynamically checks or remaps masks to the model grid as needed.
        - Processes outputs for:
            - Global (entire Earth).
            - Arctic region (latitudes above 60°N).
            - Masked regions (e.g., Arctic Ocean, permafrost land).

    """

    correction = ""  # Future placeholder for corrections if necessary

    # Build CDO command
    if dimensions == "1D":
        cdo_cmd = get_cdo_cmd_1D(variable, correction, area_file, inputfile, gridarea_var, time)
    elif dimensions == "2D":
        cdo_cmd = get_cdo_cmd_2D(variable, correction, inputfile, gridarea_var, time)
    else:
        logger.error(f"Wrong Processing description: dimension has to be either 1D or 2D but is {dimensions}")
        sys.exit()
        
    if cdo_cmd is None:
        logger.warning(f"Couldn't retrieve CDO command using get_cdo_cmd_1D or get_cdo_cmd_2D")
        return
    #logger.debug(cdo_cmd)

    # Process outputs for different regions
    def process_output(region_id: str, cdo_cmd: str, mask_cmd:str =""):
        """
        Processes output for a specified region using the provided CDO command and optional masking.
    
        Parameters:
            region_id (str): Identifier for the region being processed (e.g., "glob", "N60", "masked").
            cdo_cmd (str): CDO command string for processing the input data.
            mask_cmd (str, optional): Additional CDO command for applying a mask. Defaults to an empty string.
    
        Returns:
            None
        """
        outfile = f"{outpath}{variable}/{variable}_{region_id}_{model}_{ensemblemember}_{scenario}_{dimensions}.nc"
        if time=="mon":
            outputfile = f"{outpath}{variable}/{variable}_masked_{model}_{ensemblemember}_{scenario}_{dimensions}_mon.nc"
            
        logger.debug(f"CDO command: {mask_cmd}{cdo_cmd}")
        logger.debug("outputfile: " +outfile)
        try:
            if dimensions == "2D":
                cdo.copy(input=f"{mask_cmd}{cdo_cmd}", output=outfile, options="--reduce_dim")
            else:
                cdo.fldsum(input=f"{mask_cmd}{cdo_cmd}", output=outfile, options="--reduce_dim")
        except Exception as e:
            logger.error(f"CDO operation failed:")
            logger.error(e)
            
        file_existence(outfile, logger) # Sometimes CDO does not create a file without producing an error
        return
        
    # Global
    if earth:
        process_output("glob", cdo_cmd, "")

    # Arctic above 60°N
    if N60:
        process_output("N60", cdo_cmd, " -masklonlatbox,0,360,60,90")
    
    # Masked regions
    if masked:
        if gridarea_var == "areacello": # ocean variables
            remapped_mask = ocean_maskpath + f"{model}_arctic-ocean-mask_remapped.nc"
            mask_var      = mask_var_names["ocean"]
            mask_file     = AO_mask
        else: # areacella # land (/atm) variables
            remapped_mask = land_maskpath + f"{model}_permafrost-land-mask_remapped.nc"
            mask_var      = mask_var_names["land"]
            mask_file     = land_mask
            
        # Check if remapped mask exsists for selected model (if not remap original mask to model grid)
        if not os.path.isfile(remapped_mask):
            examplefile = glob.glob(inputfile)[0] 
            logger.debug(f"Remapping mask using {examplefile}")
            logger.debug(f"masking cmd:  -remapbil,{examplefile} -selvar,mask {mask_file} "+ remapped_mask)
            cdo.copy(input=f" -remapbil,{area_file} -selvar,{mask_var} {mask_file}", output=remapped_mask)
            file_existence(remapped_mask, logger)

        logger.debug(remapped_mask)
        process_output("masked", cdo_cmd, f" -ifthen {remapped_mask}")

## Processing Loop

In [14]:
for variable in variables[:]:
    for scenario in scenarios[:]:
        logger.info(scenario)

        # search the catalog
        activity_id, table_id, modellist, modelcenters, ensemblemembers, unit = searching.modelsearch(catalog, scenario, variable, logger, member=None)

        # set the grid area variable
        if table_id.startswith("O"): # ocean variables on the e.g. Omon or Oday table
            gridarea_var = "areacello" 
        else:
            gridarea_var = "areacella" 


        for model in tqdm(modellist[:], leave=True):#["MPI-ESM1-2-LR"]
            logger.info(model)
            
            for ensemblemember in ensemblemembers[model][:1]:
                outputfile = f"{outpath}{variable}/{variable}_masked_{model}_{ensemblemember}_{scenario}_{dimensions}.nc" 
                
                if time=="mon":
                    outputfile = f"{outpath}{variable}/{variable}_masked_{model}_{ensemblemember}_{scenario}_{dimensions}_mon.nc"
                
                if os.path.isfile(outputfile):
                    logger.debug(f"--- File already exists ({ensemblemember}).")
                    pass
                else:
                    logger.info("... "+ensemblemember)
                    inputfile, area_file = retrieve_file_names(model, modelcenters[model], ensemblemember, scenario, variable, table_id, activity_id)
                    if area_file is not None and inputfile is not None:
                        cal_annual_monthly_values(inputfile, area_file, model, ensemblemember, scenario, variable, unit, time)

VARIABLE:  fFire
100%|██████████| 14/14 [00:34<00:00,  2.46s/it]
VARIABLE:  fFire
100%|██████████| 4/4 [00:03<00:00,  1.02it/s]
VARIABLE:  fFire
100%|██████████| 5/5 [00:05<00:00,  1.10s/it]
VARIABLE:  fFire
100%|██████████| 6/6 [00:07<00:00,  1.21s/it]
VARIABLE:  fFire
100%|██████████| 5/5 [00:05<00:00,  1.07s/it]
